In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import argparse
import os

import numpy as np
import torch
from torch.nn.modules.loss import CrossEntropyLoss
from torch.utils.data.dataloader import DataLoader
from tqdm import tqdm
import random

from handwriting_recognition.label_converter import LabelConverter
from handwriting_recognition.model.model import HandwritingRecognitionModel
from handwriting_recognition.modelling_utils import get_image_model
from handwriting_recognition.utils import TrainingConfig, get_dataset_folder_path
from handwriting_recognition.dataset import HandWritingDataset
from pathlib import Path
from handwriting_recognition.modelling_utils import get_device
from handwriting_recognition.eval import cer, wer

torch.backends.cudnn.benchmark = True

/home/faraz/.cache/pypoetry/virtualenvs/handwriting-recognition-05HTZ8rb-py3.11/lib/python3.11/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [3]:
CONFIG_NAME = "default_config"
MODEL_TO_TEST = "last"
MODEL_FILE = Path(get_dataset_folder_path()).parent.joinpath("model_outputs", CONFIG_NAME, MODEL_TO_TEST)

In [4]:
config_path = Path(get_dataset_folder_path()).parent.joinpath("handwriting_recognition", "configs", CONFIG_NAME).with_suffix(".json")
config = TrainingConfig.from_path(config_path=config_path)

In [5]:
data_train = HandWritingDataset(
    data_path=get_dataset_folder_path() / "pre_processed" / "train.csv",
    img_size=config.feature_extractor_config.input_size,
)

data_val = HandWritingDataset(
    data_path=get_dataset_folder_path() / "pre_processed" / "validation.csv",
    img_size=config.feature_extractor_config.input_size,
)

data_test = HandWritingDataset(
    data_path=get_dataset_folder_path() / "pre_processed" / "test.csv",
    img_size=config.feature_extractor_config.input_size,
)

In [6]:
saved_model = torch.load(MODEL_FILE)

In [7]:
if "scheduler_config" not in saved_model['config']:
    saved_model['config']['scheduler_config'] = None
if "beta1" not in saved_model['config']['optim_config']:
    saved_model['config']['optim_config']['beta1'] = 0.95
if "beta2" not in saved_model['config']['optim_config']:
    saved_model['config']['optim_config']['beta2'] = 0.99

In [9]:
config = TrainingConfig(**saved_model['config'])

In [10]:
image_model = get_image_model(model_name=config.feature_extractor_config.model_name)
model = HandwritingRecognitionModel(image_feature_extractor=image_model, training_config=config)

In [11]:
model.load_state_dict(saved_model["state"])

<All keys matched successfully>

In [12]:
converter = LabelConverter(character_set=saved_model['character_set'], max_text_length=saved_model['max_text_length'])

In [13]:
model = model.to(get_device())
model = model.eval()

In [14]:
BATCH_SIZE = 64

In [15]:
train_loader = DataLoader(
    data_train,
    batch_size=BATCH_SIZE,
    shuffle=False,
    pin_memory=False,
    drop_last=False,
)

val_loader = DataLoader(
    data_val,
    batch_size=BATCH_SIZE,
    shuffle=False,
    pin_memory=False,
    drop_last=False,
)

test_loader = DataLoader(
    data_test,
    batch_size=BATCH_SIZE,
    shuffle=False,
    pin_memory=False,
    drop_last=False,
)

In [16]:
images, labels = next(iter(train_loader))

In [17]:
images.shape

torch.Size([64, 1, 224, 224])

In [18]:
text, length = converter.encode(labels)
text = text.to(device=get_device())
images = images.to(device=get_device())

In [19]:
preds = model(x=images, y=text[:, :-1], is_train=False)
target = text[:, 1:]

In [20]:
predicted_classes = preds.argmax(dim=-1)

In [31]:
pred_decoded = converter.decode(predicted_classes, length)
pred_decoded

['LECOURE[s][s]E[s][s][s][s][s][s][s][s][s][s]',
 'LECOURE[s][s]E[s][s][s][s][s][s][s][s][s][s]',
 'LECOURE[s][s]E[s][s][s][s][s][s][s][s][s][s]',
 'LECOURE[s][s]E[s][s][s][s][s][s][s][s][s][s]',
 'LECOURE[s][s]E[s][s][s][s][s][s][s][s][s][s]',
 'LECOURE[s][s]E[s][s][s][s][s][s][s][s][s][s]',
 'LECOURE[s][s]E[s][s][s][s][s][s][s][s][s][s]',
 'LECOURE[s][s]E[s][s][s][s][s][s][s][s][s][s]',
 'LECOURE[s][s]E[s][s][s][s][s][s][s][s][s][s]',
 'LECOURE[s][s]E[s][s][s][s][s][s][s][s][s][s]',
 'LECOURE[s][s]E[s][s][s][s][s][s][s][s][s][s]',
 'LECOURE[s][s]E[s][s][s][s][s][s][s][s][s][s]',
 'LECOURE[s][s]E[s][s][s][s][s][s][s][s][s][s]',
 'LECOURE[s][s]E[s][s][s][s][s][s][s][s][s][s]',
 'LECOURE[s][s]E[s][s][s][s][s][s][s][s][s][s]',
 'LECOURE[s][s]E[s][s][s][s][s][s][s][s][s][s]',
 'LECOURE[s][s]E[s][s][s][s][s][s][s][s][s][s]',
 'LECOURE[s][s]E[s][s][s][s][s][s][s][s][s][s]',
 'LECOURE[s][s]E[s][s][s][s][s][s][s][s][s][s]',
 'LECOURE[s][s]E[s][s][s][s][s][s][s][s][s][s]',
 'LECOURE[s][s]E[s][

In [32]:
target_decoded = converter.decode(target, length)
target_decoded

['BALTHAZAR[s][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO]',
 'SIMON[s][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO]',
 'BENES[s][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO]',
 'LA LOVE[s][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO]',
 'DAPHNE[s][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO]',
 'LUCIE[s][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO]',
 'NASSIM[s][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO]',
 'ASSRAOUI[s][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO]',
 'LAVIAN[s][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO]',
 'MAEVA[s][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO]',
 'EMMA[s][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO]',
 'MOULINIER[s][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO]',
 'ELISE[s][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO]',
 'HONNERT[s][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO]',
 'MATHEO[s][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO]',
 'PETITDIDIER[s][GO

In [23]:
cer(pred_decoded, target_decoded)

tensor(0.5926)

In [24]:
from handwriting_recognition.train import _evaluate

loss_function = CrossEntropyLoss(ignore_index=0).to(get_device())

validation_loss, val_character_error_rate, val_word_error_rate, all_val_preds, all_val_ground_truths = _evaluate(
    epoch=saved_model['epoch'],
    model=model,
    data_loader=val_loader,
    converter=converter,
    loss_function=loss_function
)

Validating Epoch: 24: 100% 624/624 [01:23<00:00,  7.46it/s]


['LECOURE', 'LECOURE', 'LECOURE', 'LECOURE', 'LECOURE', 'LECOURE', 'LECOURE', 'LECOURE', 'LECOURE', 'LECOURE']
['BILEL', 'LAUMIONIER', 'JEAN ROCH', 'RUPP', 'PICHON', 'DANIEL', 'JEREMY', 'JEAN MICHEL', 'JULIEN', 'NAEL']


In [25]:
val_character_error_rate

tensor(0.9581)

In [26]:
val_word_error_rate

tensor(1.)

In [27]:
test_loss, test_character_error_rate, test_word_error_rate, all_test_preds, all_test_ground_truths = _evaluate(
    epoch=saved_model['epoch'],
    model=model,
    data_loader=test_loader,
    converter=converter,
    loss_function=loss_function
)

Validating Epoch: 24:   0% 0/623 [00:00<?, ?it/s]

Validating Epoch: 24: 100% 623/623 [01:23<00:00,  7.47it/s]


['LECOURE', 'LECOURE', 'LECOURE', 'LECOURE', 'LECOURE', 'LECOURE', 'LECOURE', 'LECOURE', 'LECOURE', 'LECOURE']
['KEVIN', 'CLOTAIRE', 'LENA', 'JULES', 'CHERPIN', 'MARTIN', 'VALENTINE', 'LORAS', 'THIBAULT', 'AZABI']


In [28]:
test_character_error_rate

tensor(0.9605)

In [29]:
test_word_error_rate

tensor(1.)